In [1]:
import codecs, nltk, string, os, gensim
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


exclude = set(string.punctuation)

# this represent any text as a single "doc-embedding" we use it both for the query and the sentences
# input should be a string
def text_embedding(text):
    
    #you should check in the embeddings you use if the words have been lowercased or not. 
    #try ask the embedding for "barack" and for "Barack"
    # if the Barack works, then comment the following line
    text = text.lower()
    
    # we tokenize the text in single words
    text = nltk.tokenize.WordPunctTokenizer().tokenize(text)
    
    # we remove numbers and punctuation
    text = [token for token in text if token not in exclude and token.isalpha()]
    
    
    doc_embed = []
    
    # for each word we get the embedding and we append it to a list
    for word in text:
            try:
                embed_word = emb_model[word]
                doc_embed.append(embed_word)
            except KeyError:
                continue
    # we average the embeddings of all the words, getting an overall doc embedding
    if len(doc_embed)>0:
        avg = [float(sum(col))/len(col) for col in zip(*doc_embed)]

        avg = np.array(avg).reshape(1, -1)

        # the output is a doc-embedding
        return avg
    else:
        return "Empty"

C:\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
#import here your word-embeddings - put the path to the file (if it's .bin change the binary to True)
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('../../resources/small-embeddings.txt', binary=False)

# german wikipedia from https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/federiconanni/Downloads/wiki.de.vec', binary=False)
emb_model = gensim.models.KeyedVectors.load_word2vec_format('C:/Users/Dr. J/Desktop/wiki.de.vec', binary=False)

# switch for in-domain 
#emb_model = gensim.models.KeyedVectors.load_word2vec_format('C:/QTA/topic-detect/in-domain-embeddings.txt', binary=False)



In [3]:
#this can be a list of words on the same fine-grained topic, like "people", "elites"
# add more words after a space to make it more precise
# query = "volk bürger"

#focus on "key key" terms based on dictionary, let the embeddings find more 
#query = "volk"
#query = "bürger einbürgerung steuerzahler gemeinschaft"
#query = "elite politiker establishment herrschend"
#query = "korruption täuschung betrügen verrat schämen skandal wahrheit unehrlich lüge"
#query = "verantwortung glaubwürdigkeit"
#query = "souverän neutral"
#query = "demokratisch referendum volksabstimmung volksinitiative"
#query = "konsens kompromiss"
#query = "repräsentation parlament regierung "
#query = "populisten populismus demagogisch demagogen"

# populism at its best?
# query = "volk elite souverän"

# efficient flip-side query?
#query = "konsens repräsentation populismus"

query = "volk elite souverän parlament regierung repräsentation"

query_emb = text_embedding(query)

# add the path to the folder where you have your manifestos as text documents
# collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/deu2017/"

# only Germany for less output 
#collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/germany17/"

# German texts last three elections for Germany, Austria and Switzerland - 63 manifestos from polidoc.net
collection_path = "C:/Users/Dr. J/Dropbox/sparserhetoric/polidoc_allgerman/"

In [4]:
# this will be a dictionary of documents, for example manifestos, divided in sentences, which are represented as sentence embeddings
collection = {}

# you loop over the folder
for filename in os.listdir(collection_path):
    # you open each file
    # note encoding 
    content = codecs.open(collection_path+filename,"r","utf-8").read()
    # you split it in sentences
    content = nltk.sent_tokenize(content)
    
    # you represent each sentence in each document as a word-embedding, which captures the meaning of the sentence
    content = [[sent, text_embedding(sent)] for sent in content if type(text_embedding(sent))!= str]
    collection[filename] = content

In [5]:
# now, the information retrieval part

for filename,sentences in collection.items():
    
    # compare the cosine similarity between the embedding of the query and each sentence embedding
    ranking = [[sent, cosine_similarity(query_emb,sent_emb)[0][0]] for sent, sent_emb in sentences]
    # you rank them, based on the similarity
    ranking.sort(key=lambda x: x[1],reverse=True)
    
    print (filename)
    # you can change here for having more sentences as output
   # for sent, score in ranking[:20]:
    for sent, score in ranking[:3]:
        print (sent, score)
    print (" \n")

41113.000.2009.1.1.txt
Populistische Ressentiments gegen Parlamente und Parteien sind gefährlich
für unsere Demokratie. 0.7137465621228545
Anwältin der Bürgerrechte – Rechtsstaat verteidigen, Integration fördern
2009 blicken wir zurück auf 60 Jahre Grundgesetz, auf 20 Jahre friedliche Revolution in der
DDR, auf 90 Jahre Frauenwahlrecht. 0.6990308838833484
 … stärkt Parlamente und schafft mehr Transparenz in der Politik. 0.6961506290972486
 

41113.000.2013.1.1.txt
Parlamente stärken, Parteien öffnen
Die repräsentative Demokratie ist Garant für die Vertretung der gesamten Gesellschaft gegenüber Einzelinteressen. 0.7485675877306852
Unsere Demokratie braucht einen selbstbewussten Bundestag mit starken, unabhängigen Abgeordneten, die die Gesellschaft abbilden und dem Drängen mächtiger Interessen standhalten. 0.7233755266835471
Dazu fordern wir einen europäisierten und personalisierten Wahlkampf zum Europäischen Parlament ebenso wie mehr Mitspracherechte der Bürgerinnen und Bürger. 0.71681

41950.000.2013.1.1.txt
Demokratie Add-on für Europa
Wir Piraten fordern die Einberufung einer verfassungsgebenden Versammlung (Verfassungskonvent) für die Europäische Union. 0.7118954525883217
Die aktuelle Gesetzgebung der EU wird von der Exekutive, der Europäischen Kommission, zu Lasten der eigentlichen Legislative, dem Europäischen Parlament, dominiert. 0.7092590656052649
Jede Erhöhung des Stammkapitals soll von den demokratisch gewählten Volksvertretern der jeweiligen nationalen Parlamente beschlossen werden. 0.7084498067109595
 

41953.000.2013.1.1.txt
Jedes Volk muss
demokratisch über seine Währung entscheiden dürfen. 0.7209739824874661
- Wir bestehen auf dem uneingeschränkten Budgetrecht der nationalen
Parlamente. 0.7102531853731473
Das Volk
soll den Willen der Parteien bestimmen, nicht
umgekehrt. 0.6851251091713991
 

41953.000.2017.1.1.txt
- Das Volk muss wieder zum Souverän werden
Heimlicher Souverän in Deutschland ist eine kleine, machtvolle politische Oligarchie, die sich in

42951.000.2008.1.1.txt
In dieser Situation hat der Autor und unabhängige EU-Parlamentarier Dr. Hans-Peter Martin eine „Bürgerliste für Demokratie, Kontrolle, Gerechtigkeit“ gegründet und engagierte Bürgerinnen und Bürger zur Kandidatur bei den anstehenden Nationalratswahlen am 1. 0.6781895626791028
In diesem Sinne streben wir mit größtmöglicher Kraft eine Änderung der österreichischen
Bundesverfassung an und fordern:
- ein Rederecht einer Auswahl von direkt betroffenen Bürgern zu geplanten
Gesetzesbeschlüssen in den Sitzungen des Nationalrates
- die Veröffentlichung von Gesetzesbeschlüssen noch vor Inkrafttreten (auch im
Internet)
- die Stärkung des Gesetzesinitiativrechtes der Bürger (Volksbegehren) durch
zwingende Volksabstimmung ab 500.000 Unterstützer, wenn der Nationalrat
Gesetzesinitiative nicht umsetzt
- eine zwingende Volksabstimmung über einen Gesetzesbeschluss (über die Fälle der
Gesamtänderung der Verfassung hinaus) noch vor Inkrafttreten, wenn dies 500.000
Bürger verlangen


43810.000.2007.1.1.txt
Die vier Säulen unserer politischen Identität – Unabhängigkeit, Föderalismus,
Volksrechte und immerwährende, integrale, bewaffnete Neutralität 
– müssten preisgegeben werden. 0.7103628575439994
Abkehr vom Erfolgsrezept verhindern
Die bisherige Geschichte hat gezeigt, dass ein unabhängiger,
föderalistischer, direktdemokratischer Staat auf der Basis
einer liberalen Wirtschaftsordnung den Menschen Wohlstand,
Freiheit und Sicherheit bieten kann. 0.7049532451477811
Dazu gehören die immerwährende, integrale und bewaffnete Neutralität, die direkte Demokratie, der Föderalismus, die Gemeindeautonomie, die eigenständige Währung und die Steuerhoheit. 0.6999421281447009
 

43810.000.2011.1.1.txt
Die Bevölkerung wird ihre Regierung im
Gegensatz zum Parlament ohne eigennützige, kleinkarierte
Ränkespiele, parteipolitische Intrigen und Hintertreppenabsprachen
wählen. 0.7727642442750039
Bundesrat, Parlament und Verwaltung
vernachlässigen das eigene Land. 0.7521931787464333
Uns ei